### Importando Librerías

In [23]:
import pandas as pd
import numpy as np

### Leo archivo p_data.csv y seteo el index por ID

In [13]:
dfp_Data = pd.read_csv('files/p_data.csv', sep=';')
dfp_Data = dfp_Data.set_index('ID')
print(dfp_Data.shape)
dfp_Data.head()

(48842, 8)


,Age,Sex,Race,Country,Education,Education-Num,Marital Status,Relationship
ID,,,,,,,,
49084,39.0,Male,White,United-States,Bachelors,13.0,Never-married,Not-in-family
68553,50.0,Male,White,United-States,Bachelors,13.0,Married-civ-spouse,Husband
143654,38.0,Male,White,United-States,HS-grad,9.0,Divorced,Not-in-family
41510,53.0,Male,Black,United-States,11th,7.0,Married-civ-spouse,Husband
195025,28.0,Female,Black,Cuba,Bachelors,13.0,Married-civ-spouse,Wife


### Leo archivo w_data y seteo el index por ID

In [15]:
dfw_Data = pd.read_csv('files/w_data.csv', sep=';')
dfw_Data = dfw_Data.set_index('ID')
print(dfw_Data.shape)
dfw_Data.head()

(48843, 6)


,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,Target
ID,,,,,,
49084,State-gov,77516.0,2174.0,0.0,40.0,<=50K
68553,Self-emp-not-inc,83311.0,0.0,0.0,13.0,<=50K
143654,Private,215646.0,0.0,0.0,40.0,<=50K
41510,Private,234721.0,0.0,0.0,40.0,<=50K
195025,Private,338409.0,0.0,0.0,40.0,<=50K


### Fusiono los dos data frames en función del ID

In [19]:
dfP_W = pd.merge(dfp_Data, dfw_Data, how='outer', on='ID')
print(dfP_W.shape)
dfP_W.head()

(48843, 14)


,Age,Sex,Race,Country,Education,Education-Num,Marital Status,Relationship,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,Target
ID,,,,,,,,,,,,,,
49084,39.0,Male,White,United-States,Bachelors,13.0,Never-married,Not-in-family,State-gov,77516.0,2174.0,0.0,40.0,<=50K
68553,50.0,Male,White,United-States,Bachelors,13.0,Married-civ-spouse,Husband,Self-emp-not-inc,83311.0,0.0,0.0,13.0,<=50K
143654,38.0,Male,White,United-States,HS-grad,9.0,Divorced,Not-in-family,Private,215646.0,0.0,0.0,40.0,<=50K
41510,53.0,Male,Black,United-States,11th,7.0,Married-civ-spouse,Husband,Private,234721.0,0.0,0.0,40.0,<=50K
195025,28.0,Female,Black,Cuba,Bachelors,13.0,Married-civ-spouse,Wife,Private,338409.0,0.0,0.0,40.0,<=50K


In [27]:
dfp_Data['Age'].isnull().sum()
dfP_W['Age'].isnull().sum()

2